# 임베딩 기반의 연관어 추출

이 노트북에서는 전처리된 YouTube 데이터를 활용하여 **Word2Vec** 기반으로 연관어를 추출하는 과정을 설명합니다. 

데이터는 사전에 수집 및 전처리된 유튜브 비디오 메타 데이터를 사용하며, 주로 비디오 조회수와 키워드를 중심으로 분석을 진행합니다. **Word2Vec** 모델은 해당 키워드들의 의미적 유사성을 학습하여 연관된 단어들을 벡터 공간에서 가까운 위치에 배치합니다. 이를 통해 특정 키워드에 대한 연관어를 효과적으로 추출하고자 합니다.

## 주요 단계:
1. **데이터 로드**: 수집된 유튜브 데이터의 정제 및 전처리된 데이터 불러오기.
2. **Word2Vec 학습**: 전처리된 텍스트 데이터를 바탕으로 임베딩 벡터 학습.
3. **연관어 추출**: 학습된 모델을 활용하여 특정 키워드에 대한 유사도 높은 연관어 목록 도출.
4. **결과 분석**: 추출된 연관어의 의미적 유사성 및 활용 가능성 검토.

In [2]:
import os
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

In [3]:
# 학습 데이터 로드
df = pd.read_csv("../data/video_sample_20241013.csv", encoding="utf-8-sig", index_col=0)
df.use_text = df.use_text.apply(lambda x: [word.strip() for word in x.split(",")])
train_data = df.use_text.tolist()

In [4]:
# 하이퍼파라미터 설정
vector_size = 200
min_count = 5
window = 3
workers = 18
epochs = 1000
epoch_increment = 100
patience = 5

In [5]:
# 학습 관련 변수 초기화
previous_losses = []  # 각 epoch에서 손실값을 저장할 리스트
patience_counter = 0  # 개선이 없는 epoch 횟수를 추적하는 카운터
best_loss = np.inf  # 현재까지의 최소 손실값을 저장 (초기값은 무한대)

# Word2Vec 모델 초기화
model = Word2Vec(train_data, vector_size=vector_size, 
                window=window, min_count=min_count, 
                workers=workers, sg=0, compute_loss=True)  # sg=0은 CBOW, compute_loss=True로 손실값 계산 활성화

# 점진적 학습 루프
for epoch in range(0, epochs, epoch_increment):  # 주어진 epoch 범위에서 epoch_increment 단위로 학습 진행
    model.train(train_data, total_examples=model.corpus_count, epochs=epoch_increment)  # Word2Vec 모델 학습
    current_loss = model.get_latest_training_loss()  # 현재 학습 손실값을 가져옴

    # 손실값 평가
    if current_loss < best_loss:  # 이전 최저 손실값보다 현재 손실값이 더 작으면
        best_loss = current_loss  # 최저 손실값 갱신
        patience_counter = 0  # 손실값이 개선되었으므로 카운터 초기화
    else:
        patience_counter += 1  # 손실값이 개선되지 않으면 카운터 증가

    # 손실값을 저장하고 확인
    previous_losses.append(current_loss)  # 현재 손실값을 리스트에 저장
    print(f"Epoch: {epoch + epoch_increment}, Loss: {current_loss}")  # 현재 epoch과 손실값 출력

    # 연속 5번 손실값이 줄어들지 않으면 학습 중단
    if patience_counter >= patience:  # 설정된 patience 값(여기선 5) 이상 개선되지 않으면 조기 종료
        print("Early stopping: No improvement in loss for 5 consecutive epochs.")  # 조기 종료 메시지 출력
        break  # 학습 종료


Epoch: 100, Loss: 0.0
Epoch: 200, Loss: 0.0
Epoch: 300, Loss: 0.0
Epoch: 400, Loss: 0.0
Epoch: 500, Loss: 0.0
Epoch: 600, Loss: 0.0
Early stopping: No improvement in loss for 5 consecutive epochs.


In [6]:
# 모델 저장
cache_path = "../models"
filepath = os.path.join(cache_path, "related_model.bin")
model.save(filepath)

In [7]:
# 연관어 추출
keyword = "이강인"
model.wv.similar_by_word(keyword)


[('김민재', 0.5350412130355835),
 ('레길론', 0.5053242444992065),
 ('아드리아누', 0.5034506320953369),
 ('피를로', 0.48978903889656067),
 ('황의조', 0.4843679368495941),
 ('손흥민', 0.4828548729419708),
 ('손홍민', 0.4817011058330536),
 ('세자르', 0.4740636646747589),
 ('조규성', 0.4608883261680603),
 ('축구대표팀', 0.45687970519065857)]